In [8]:
import numpy as np
import pandas as pd
import polars as pl

In [2]:
!wget https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar.vcf.gz

--2024-09-02 11:55:23--  https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar.vcf.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.12, 130.14.250.31, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103478058 (99M) [application/x-gzip]
Saving to: ‘clinvar.vcf.gz’

clinvar.vcf.gz      100%[===================>]  98.68M  28.0MB/s    in 3.7s    

2024-09-02 11:55:28 (26.5 MB/s) - ‘clinvar.vcf.gz’ saved [103478058/103478058]



In [4]:
!zcat clinvar.vcf.gz | head -n 30

##fileformat=VCFv4.1
##fileDate=2024-08-25
##source=ClinVar
##reference=GRCh38
##ID=<Description="ClinVar Variation ID">
##INFO=<ID=AF_ESP,Number=1,Type=Float,Description="allele frequencies from GO-ESP">
##INFO=<ID=AF_EXAC,Number=1,Type=Float,Description="allele frequencies from ExAC">
##INFO=<ID=AF_TGP,Number=1,Type=Float,Description="allele frequencies from TGP">
##INFO=<ID=ALLELEID,Number=1,Type=Integer,Description="the ClinVar Allele ID">
##INFO=<ID=CLNDN,Number=.,Type=String,Description="ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB">
##INFO=<ID=CLNDNINCL,Number=.,Type=String,Description="For included Variant : ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB">
##INFO=<ID=CLNDISDB,Number=.,Type=String,Description="Tag-value pairs of disease database name and identifier submitted for germline classifications, e.g. OMIM:NNNNNN">
##INFO=<ID=CLNDISDBINCL,Number=.,Type=String,Description="For 

In [7]:
from cyvcf2 import VCF

rows = []
for variant in VCF("clinvar.vcf.gz"):
    if variant.INFO.get("CLNVC") != "single_nucleotide_variant": continue
    if len(variant.ALT) != 1: continue
    cln_sig = variant.INFO.get("CLNSIG")
    if cln_sig not in ["Pathogenic", "Likely_Pathogenic"]: continue
    MC = variant.INFO.get("MC")
    if MC is None: continue
    consequences = [x.split("|")[1] for x in MC.split(",")]
    consequence = ','.join(np.unique(consequences))
    review_status = variant.INFO.get("CLNREVSTAT")
    rows.append([variant.CHROM, variant.POS, variant.REF, variant.ALT[0], cln_sig, variant.ID, review_status, consequence])

V = pd.DataFrame(rows, columns=["chrom", "pos", "ref", "alt", "label", "id", "review_status", "consequence"])
V = V[V.chrom!="MT"]
V = V[V.alt != "N"]
V

[W::vcf_parse] Contig '1' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '2' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '3' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '4' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '5' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '6' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '7' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '8' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '9' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '10' is not defined in the header. (Quick workaroun

NameError: name 'pd' is not defined

In [10]:
V

,chrom,pos,ref,alt,label,id,review_status,consequence
0,1,943995,C,T,Pathogenic,950448,"criteria_provided,_single_submitter",nonsense
1,1,976215,A,G,Pathogenic,1320032,no_assertion_criteria_provided,missense_variant
2,1,1014143,C,T,Pathogenic,183381,no_assertion_criteria_provided,nonsense
3,1,1022368,C,A,Pathogenic,3255595,"criteria_provided,_single_submitter",nonsense
4,1,1041582,C,T,Pathogenic,126556,no_assertion_criteria_provided,nonsense
...,...,...,...,...,...,...,...,...
75743,Y,2787515,C,A,Pathogenic,492908,no_assertion_criteria_provided,missense_variant
75744,Y,2787551,C,T,Pathogenic,9754,no_assertion_criteria_provided,missense_variant
75745,Y,2787592,A,T,Pathogenic,9751,no_assertion_criteria_provided,nonsense
75746,Y,2787600,G,A,Pathogenic,9753,no_assertion_criteria_provided,nonsense


In [14]:
V2 = V[
    ~(V.consequence.str.contains("missense")) &
    ~(V.consequence.str.contains("coding")) &
    ~(V.consequence.str.contains("nonsense")) &
    ~(V.consequence.str.contains("UTR"))
]
V2

,chrom,pos,ref,alt,label,id,review_status,consequence
8,1,1043197,G,C,Pathogenic,1284256,no_assertion_criteria_provided,intron_variant
40,1,1787438,C,A,Pathogenic,1315579,no_assertion_criteria_provided,splice_acceptor_variant
41,1,1789052,C,A,Pathogenic,2703061,"criteria_provided,_single_submitter",splice_donor_variant
48,1,1804582,C,A,Pathogenic,1315576,no_assertion_criteria_provided,splice_acceptor_variant
81,1,2408451,C,T,Pathogenic,6770,"criteria_provided,_multiple_submitters,_no_con...",splice_donor_variant
...,...,...,...,...,...,...,...,...
75683,X,154984804,C,T,Pathogenic,1698780,"criteria_provided,_single_submitter",splice_acceptor_variant
75684,X,154984805,T,C,Pathogenic,10182,no_assertion_criteria_provided,splice_acceptor_variant
75687,X,154987307,T,C,Pathogenic,10162,no_assertion_criteria_provided,splice_acceptor_variant
75697,X,154996972,C,T,Pathogenic,1330755,"criteria_provided,_single_submitter",splice_donor_variant


In [15]:
V2.consequence.value_counts()

consequence
splice_donor_variant                                                                 5223
splice_acceptor_variant                                                              3961
intron_variant                                                                       1294
intron_variant,splice_donor_variant                                                   505
intron_variant,splice_acceptor_variant                                                419
synonymous_variant                                                                    225
stop_lost                                                                              49
intron_variant,synonymous_variant                                                      24
splice_donor_variant,synonymous_variant                                                 7
splice_acceptor_variant,synonymous_variant                                              5
intron_variant,stop_lost                                                                